In [ ]:
#数据质量
# 一 有效性：审核有效性是关于确定对个别字段施加什么约束并检查以确保字段值遵守这些约束条件
# 跨字段限制：当一个数据项拥有多个字段时，它们必须在某种程度上具有一致性
#准确率
#完整性
#一致性
#统一性

#使用蓝图的示例
#!/usr/bin/env python
          # -*- coding: utf-8 -*-
          import xml.etree.cElementTree as ET
          from collections import defaultdict
          import re

          osm_file = open("chicago.osm", "r")

          street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
          street_types = defaultdict(int)

          def audit_street_type(street_types, street_name):
              m = street_type_re.search(street_name)
              if m:
                  street_type = m.group()

                  street_types[street_type] += 1

          def print_sorted_dict(d):
              keys = d.keys()
              keys = sorted(keys, key=lambda s: s.lower())
              for k in keys:
                  v = d[k]
                  print "%s: %d" % (k, v) 

          def is_street_name(elem):
              return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

          def audit():
              for event, elem in ET.iterparse(osm_file):
                  if is_street_name(elem):
                      audit_street_type(street_types, elem.attrib['v'])    
              print_sorted_dict(street_types)    

          if __name__ == '__main__':
              audit()

In [1]:
#练习: 修正有效性
# Your task is to check the "productionStartYear" of the DBPedia autos datafile for valid values.
# The following things should be done:
# - check if the field "productionStartYear" contains a year
# - check if the year is in range 1886-2014
# - convert the value of the field to be just a year (not full datetime)
# - the rest of the fields and values should stay the same
# - if the value of the field is a valid year in the range as described above,
#   write that line to the output_good file
# - if the value of the field is not a valid year as described above, 
#   write that line to the output_bad file
# - discard rows (neither write to good nor bad) if the URI is not from dbpedia.org
# - you should use the provided way of reading and writing data (DictReader and DictWriter)
#   They will take care of dealing with the header.
# 你的任务是检查 DBPedia 自动数据文件的“productionStartYear”并获取有效的值。应该完成以下任务：
# -    检查字段“productionStartYear”是否包含年份
# -    检查该年份是否在 1886 至 2014 范围内
# -    将字段值转换为年份（而不是整个日期时间）
# -    字段的其他部分和值应该保持不变
# -    如果字段的值是如上所述范围内的有效年份，则将该行写入 output_good 文件中
# -    如果字段的值不是如上所述的有效年份，则将该行写入 output_bad 文件中
# -    你应该采用提供的数据读取和写入方式（DictReader 和 DictWriter），它们将会对标题进行处理。
# 
# You can write helper functions for checking the data and writing the files, but we will call only the 
# 'process_file' with 3 arguments (inputfile, output_good, output_bad).
# 你可以编写辅助函数来检查数据并编写文件，
# 但是我们将仅调用包含三个参数（inputfile、output_good、output_bad）的“process_file”文件。

import csv
import pprint
from datetime import datetime
import re

INPUT_FILE = 'autos.csv'
OUTPUT_GOOD = 'autos-valid.csv'
OUTPUT_BAD = 'FIXME-autos.csv'
good = []
bad = []

def process_file(input_file, output_good, output_bad):

    # with open(input_file, "r") as f:
    #     reader = csv.DictReader(f)
    #     header = reader.fieldnames
    #     print header
    
    with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        # good.append(header)
        # bad.append(header)
        p = re.compile('\d{4}', re.IGNORECASE)
        for row in reader:
            if row['URI'].find("dbpedia.org") < 0:
                continue
            m =re.search('\d{4}',row['productionStartYear'])
            if m:
                year = int(m.group())
                if year > 1886 and year <  2014:
                     row['productionStartYear'] = year
                     good.append(row)
                else:
                    row['productionStartYear'] = year
                    bad.append(row)
                    
            else:
                bad.append(row)
        

        #COMPLETE THIS FUNCTION



    # This is just an example on how you can use csv.DictWriter
    # Remember that you have to output 2 files
    
    with open(output_good, "w") as g:
        writer = csv.DictWriter(g, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in good:
            writer.writerow(row)
            
            
    with open(output_bad, "w") as g:
        writer = csv.DictWriter(g, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in bad:
            writer.writerow(row)
            


def test():

    process_file(INPUT_FILE, OUTPUT_GOOD, OUTPUT_BAD)
    


    
        # print datetime.strptime(row['productionStartYear'],"%Y%d%m")
        
        
    
test()

# ANSWER:
# def process_file(input_file, output_good, output_bad):
#     # store data into lists for output
#     data_good = []
#     data_bad = []
#     with open(input_file, "r") as f:
#         reader = csv.DictReader(f)
#         header = reader.fieldnames
#         for row in reader:
#             # validate URI value
#             if row['URI'].find("dbpedia.org") < 0:
#                 continue
# 
#             ps_year = row['productionStartYear'][:4]
#             try: # use try/except to filter valid items
#                 ps_year = int(ps_year)
#                 row['productionStartYear'] = ps_year
#                 if (ps_year >= 1886) and (ps_year <= 2014):
#                     data_good.append(row)
#                 else:
#                     data_bad.append(row)
#             except ValueError: # non-numeric strings caught by exception
#                 if ps_year == 'NULL':
#                     data_bad.append(row)
# 
#     # Write processed data to output files
#     with open(output_good, "w") as good:
#         writer = csv.DictWriter(good, delimiter=",", fieldnames= header)
#         writer.writeheader()
#         for row in data_good:
#             writer.writerow(row)
# 
#     with open(output_bad, "w") as bad:
#         writer = csv.DictWriter(bad, delimiter=",", fieldnames= header)
#         writer.writeheader()
#         for row in data_bad:
#             writer.writerow(row)

In [130]:
#练习: 审查数据质量
# In this problem set you work with cities infobox data, audit it, come up with a
# cleaning idea and then clean it up. In the first exercise we want you to audit
# the datatypes that can be found in some particular fields in the dataset.
# The possible types of values can be:
# - NoneType if the value is a string "NULL" or an empty string ""
# - list, if the value starts with "{"
# - int, if the value can be cast to int
# - float, if the value can be cast to float, but CANNOT be cast to int.
#    For example, '3.23e+07' should be considered a float because it can be cast
#    as float but int('3.23e+07') will throw a ValueError
# - 'str', for all other values
# 在此习题集中，你将处理城市 infobox 数据，对数据进行审核，然后想出清理方法并清理数据。在第一道练习中，请审核
# 数据集中某些特定字段中的数据类型。
# 值类型可以是：
# -    NoneType，如果值是字符串“NULL”或空字符串“”
# -    列表，如果值以“{”开头
# -    整型，如果值可以转型为整型
# -    浮点型，如果值可以转型为浮点型，但是无法转型为整型。
# 例如，“3.23e+07”应该被当做浮点型，因为可以转型为浮点型，但是int('3.23e+07') 将抛出 ValueError
# -    “str”，表示其他所有值
# 
# The audit_file function should return a dictionary containing fieldnames and a 
# SET of the types that can be found in the field. e.g.
# {"field1": set([type(float()), type(int()), type(str())]),
#  "field2": set([type(str())]),
#   ....
# }
# The type() function returns a type object describing the argument given to the 
# function. You can also use examples of objects to create type objects, e.g.
# type(1.1) for a float: see the test function below for examples.
# audit_file 函数应该返回一个字典，其中包含字段名称和可以在该字段中找到的类型集。例如
# {"field1": set([type(float()), type(int()), type(str())]),
#  "field2": set([type(str())]),
#   ....
# }
# type() 函数返回的是类型对象，描述了提供给该函数的参数。你还可以使用对象示例创建类型对象，例如 type(1.1) 
# 表示浮点型：具体示例请参阅下面的测试函数。
# 
# Note that the first three rows (after the header row) in the cities.csv file
# are not actual data points. The contents of these rows should note be included
# when processing data types. Be sure to include functionality in your code to
# skip over or detect these rows.
# 注意，cities.csv 文件的前三行（标题行之后）不是实际的数据点。在处理数据类型时，不应该包含这些行的内容。
# 确保在代码中包含相关功能，以便跳过或检测出这些行。

# <type 'float'>
# <type 'str'>
# <type 'int'>
# <type 'str'>
# <type 'NoneType'>

import codecs
import csv
import json
import pprint

CITIES = 'cities.csv'

# FIELDS = ["areaLand", "areaMetro"]
FIELDS = ["name", "timeZone_label", "utcOffset", "homepage", "governmentType_label",
          "isPartOf_label", "areaCode", "populationTotal", "elevation",
          "maximumElevation", "minimumElevation", "populationDensity",
          "wgs84_pos#lat", "wgs84_pos#long", "areaLand", "areaMetro", "areaUrban"]

def skip_lines(input_file, skip):
    for i in range(0,skip):
        next(input_file)

def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def is_int(s):
    try:
        int(s)
        return True
    except ValueError:
        return False
    
def is_null(s):
    if s == 'NULL' or s == '':
        return True
    else:
        return False
        # s is None or s == ''
       

def is_list(s):
    if s[0] == '{':
        # s.startwith('{')
       
        return True
    else:
        return False


    
def audit_file(filename, fields):
    
    fieldtypes = {}
    for field in fields:
        fieldtypes[field] = set()
        
    
    with open(filename,'r') as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        print header
        skip_lines(reader,3)
    
    # for field in fields:
    #     fieldtypes[field] = set()
        for row in reader:
            for field in fields:
                # if is_null(row[field]):
                #     print 'Y'
                
                if is_null(row[field]):
                    fieldtypes[field].add(type(None))
                elif is_float(row[field]):
                    fieldtypes[field].add(type(float()))
                elif is_int(row[field]):
                    fieldtypes[field].add(type(int()))
                elif is_list(row[field]):
                    fieldtypes[field].add(type(list()))
                # elif is_null(row[field]):
                #     print 'Y'
                #     fieldtypes[field].add(type(None))
                else:
                    fieldtypes[field].add(type(str()))
         
            
    # print header
        
            
    
    # YOUR CODE HERE

    
    return fieldtypes


    
# def test():
#     fieldtypes = audit_file(CITIES, FIELDS)
# 
#     pprint.pprint(fieldtypes)
# 
#     assert fieldtypes["areaLand"] == set([type(1.1), type([]), type(None)])
#     assert fieldtypes['areaMetro'] == set([type(1.1), type(None)])
#     
# if __name__ == "__main__":
#     test()

audit_file(CITIES,FIELDS)

['URI', 'rdf-schema#label', 'rdf-schema#comment', 'administrativeDistrict_label', 'administrativeDistrict', 'anthem_label', 'anthem', 'area', 'areaCode', 'areaLand', 'areaMetro', 'areaRural', 'areaTotal', 'areaUrban', 'areaWater', 'city_label', 'city', 'code', 'country_label', 'country', 'daylightSavingTimeZone_label', 'daylightSavingTimeZone', 'district_label', 'district', 'division_label', 'division', 'elevation', 'federalState_label', 'federalState', 'foundingDate', 'foundingPerson_label', 'foundingPerson', 'foundingYear', 'governingBody_label', 'governingBody', 'government_label', 'government', 'governmentType_label', 'governmentType', 'isPartOf_label', 'isPartOf', 'isoCodeRegion_label', 'isoCodeRegion', 'leader_label', 'leader', 'leaderName_label', 'leaderName', 'leaderParty_label', 'leaderParty', 'leaderTitle', 'location_label', 'location', 'maximumElevation', 'mayor_label', 'mayor', 'minimumElevation', 'motto', 'municipality_label', 'municipality', 'part_label', 'part', 'percent

{'areaCode': {float, list, NoneType, str},
 'areaLand': {float, list, NoneType},
 'areaMetro': {float, list, NoneType},
 'areaUrban': {float, list, NoneType},
 'elevation': {float, list, NoneType},
 'governmentType_label': {list, NoneType, str},
 'homepage': {list, NoneType, str},
 'isPartOf_label': {list, NoneType, str},
 'maximumElevation': {float, list, NoneType},
 'minimumElevation': {float, NoneType},
 'name': {list, NoneType, str},
 'populationDensity': {float, list, NoneType},
 'populationTotal': {float, list, NoneType},
 'timeZone_label': {list, NoneType, str},
 'utcOffset': {float, int, list, NoneType, str},
 'wgs84_pos#lat': {float, list, NoneType},
 'wgs84_pos#long': {float, list, NoneType}}

In [15]:

print is_list('9')
def is_list(s):
    if s[0] == '{':
        # s.startwith('{')
       
        return True
    else:
        return False

False


In [137]:
# 练习: 修复区域
# In this problem set you work with cities infobox data, audit it, come up with a
# cleaning idea and then clean it up.
# 在此习题集中，你将处理城市 infobox 数据，对数据进行审核，然后想出清理方法并清理数据。
# 
# Since in the previous quiz you made a decision on which value to keep for the
# "areaLand" field, you now know what has to be done.
# 因为在上一道测验中，你已经决定针对“areaLand”字段保留哪个值，你现在知道该如何操作了。
# 
# Finish the function fix_area(). It will receive a string as an input, and it
# has to return a float representing the value of the area or None.
# You have to change the function fix_area. You can use extra functions if you
# like, but changes to process_file will not be taken into account.
# The rest of the code is just an example on how this function can be used.
# 完成函数 fix_area()。它将获得字符串输入，并需要返回表示面积值的浮点值或 None。你需要更改fix_area 函数。
# 你可以使用其他函数，但是对 process_file 的更改不会计入评估范围。
# 代码的其余部分只是用来展示可以如何使用该函数的示例。

import codecs
import csv
import json
import pprint
import decimal

# CITIES = 'cities.csv'
CITIES = 'cities_test.csv'


def fix_area(area):
    
    if is_list(area):
        area = area.strip('{').strip('}')
        area_list = area.split('|')
        s1 = area_list[0]
        s2 = area_list[1]
        if area.find('e') == -1:
           s1 = '%1e' % Decimal(s1)
           s2 = '%1e' % Decimal(s2)
        
        if s1.find('e') > s2.find('e'):
            return float(area_list[0])
        else:
            return float(area_list[1])
    elif is_null(area):
        return None
    else:
        return float(area)
    
        
    # YOUR CODE HERE

    # return area



def process_file(filename):
    # CHANGES TO THIS FUNCTION WILL BE IGNORED WHEN YOU SUBMIT THE EXERCISE
    data = []

    with open(filename, "r") as f:
        reader = csv.DictReader(f)

        #skipping the extra metadata
        for i in range(3):
            l = reader.next()

        # processing file
        for line in reader:
            # calling your function to fix the area value
            
            if "areaLand" in line:
                line["areaLand"] = fix_area(line["areaLand"])
            data.append(line)
           
 
            
    return data


def test():
    data = process_file(CITIES)

    print "Printing three example results:"
    for n in range(5,8):
        pprint.pprint(data[n]["areaLand"])
  
    assert data[3]["areaLand"] == None        
    assert data[8]["areaLand"] == 55166700.0
    assert data[20]["areaLand"] == 14581600.0
    assert data[33]["areaLand"] == 20564500.0    


# if __name__ == "__main__":
#     test()



In [149]:
# 练习: 修复姓名
# 
# In this problem set you work with cities infobox data, audit it, come up with a
# cleaning idea and then clean it up.
# 在此习题集中，你将处理城市 infobox 数据，对数据进行审核，然后想出清理方法并清理数据。
# 
# In the previous quiz you recognized that the "name" value can be an array (or
# list in Python terms). It would make it easier to process and query the data
# later if all values for the name are in a Python list, instead of being
# just a string separated with special characters, like now.
# 在上一道测验中，你意识到“name”值可以是数组（或用 Python 术语来说的话，是列表）。
# 如果名称的所有值是 Python 列表（而不是用特殊字符分隔的字符串，例如现在的状况），则稍后更容易处理和查询数据。
# 
# Finish the function fix_name(). It will recieve a string as an input, and it
# will return a list of all the names. If there is only one name, the list will
# have only one item in it; if the name is "NULL", the list should be empty.
# The rest of the code is just an example on how this function can be used.
# 请完成函数fix_name()。它将获得字符串输入，并返回所有名称列表。
# 如果只有一个名称，列表将只有一项。如果名称是“NULL”，该列表应该为空。
# 代码的其余部分只是可以用来展示如何使用该函数的示例。

import codecs
import csv
import pprint

CITIES = 'cities_test.csv'


def fix_name(name):
    if is_list(name):
        name = name.strip('{').strip('}')
        name = name.split('|')
    elif is_null(name):
        name = []
    else:
        name = [name]
   
    # YOUR CODE HERE

    return name


def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        #skipping the extra metadata
        for i in range(3):
            l = reader.next()
        # processing file
        for line in reader:
            
            
            if "name" in line:
                line["name"] = fix_name(line["name"])
            data.append(line)
            # print line["name"] 
    return data


def test():
    data = process_file(CITIES)

    print "Printing 20 results:"
    for n in range(20):
        pprint.pprint(data[n]["name"])

    assert data[14]["name"] == ['Negtemiut', 'Nightmute']
    assert data[9]["name"] == ['Pell City Alabama']
    assert data[3]["name"] == ['Kumhari']

if __name__ == "__main__":
    test()

# process_file(CITIES)

Printing 20 results:
['Kud']
['Kuju']
['Kumbhraj']
['Kumhari']
['Kunigal']
['Kurgunta']
['Athens']
['Demopolis']
['Chelsea Alabama']
['Pell City Alabama']
['City of Northport']
['Sand Point']
['Unalaska Alaska']
['City of Menlo Park']
['Negtemiut', 'Nightmute']
['Fairbanks Alaska']
['Homer']
['Ketchikan Alaska']
['Nuniaq', 'Old Harbor']
['Rainier Washington']


In [156]:
# 练习: 交叉字段审查
# In this problem set you work with cities infobox data, audit it, come up with a
# cleaning idea and then clean it up.
# 在此习题集中，你将处理城市 infobox 数据，对数据进行审核，然后想出清理方法并清理数据。
# 
# If you look at the full city data, you will notice that there are couple of
# values that seem to provide the same information in different formats: "point"
# seems to be the combination of "wgs84_pos#lat" and "wgs84_pos#long". However,
# we do not know if that is the case and should check if they are equivalent.
# 如果你查看完整的城市数据，会发现有几个值似乎提供的是同一信息，只是格式不同：
# “point”似乎是“wgs84_pos#lat”和“wgs84_pos#long”的结合体。但是，我们不知道是否是这种情况，
# # 你应该检查它们是否相等。
# 
# Finish the function check_loc(). It will recieve 3 strings: first, the combined
# value of "point" followed by the separate "wgs84_pos#" values. You have to
# extract the lat and long values from the "point" argument and compare them to
# the "wgs84_pos# values, returning True or False.
# 完成函数check_loc()。它应该获得 3 个字符串：首先是“point”的值后面跟上单独的“wgs84_pos#”值。
# # 你应该从“point”参数中提取lat和长值，并将它们与“wgs84_pos#”值对比，返回 True 或 False。
# 
# Note that you do not have to fix the values, only determine if they are
# consistent. To fix them in this case you would need more information. Feel free
# to discuss possible strategies for fixing this on the discussion forum.
# 注意，你不需要修正这些值，只需判断它们是否保持一致。要修正这些值，你需要更多信息。
# 欢迎在论坛上讨论如何修正这些值。
# 
# The rest of the code is just an example on how this function can be used.
# Changes to "process_file" function will not be taken into account for grading.
# 代码的其余部分只是可以用来展示如何使用该函数的示例。我们在打分时，不会考虑对“process_file”函数做出的更改。

import csv
import pprint

# CITIES = 'cities.csv'
CITIES = 'cities_test.csv'


def check_loc(point, lat, longi):
    # YOUR CODE HERE
    point = point.split()
    if point[0] == lat and point[1] == longi:
        
        return True
    else:
        return False
    
    
    pass


def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        #skipping the extra matadata
        for i in range(3):
            l = reader.next()
        # processing file
        for line in reader:
            # print line["point"].split(),": {",line["wgs84_pos#lat"],": ",line["wgs84_pos#long"]," }"
            # calling your function to check the location
            result = check_loc(line["point"], line["wgs84_pos#lat"], line["wgs84_pos#long"])
            if not result:
                print "{}: {} != {} {}".format(line["name"], line["point"], line["wgs84_pos#lat"], 
                                               line["wgs84_pos#long"])
            data.append(line)

    return data


def test():
    assert check_loc("33.08 75.28", "33.08", "75.28") == True
    assert check_loc("44.57833333333333 -91.21833333333333", "44.5783", "-91.2183") == False

# if __name__ == "__main__":
#     test()

process_file(CITIES)

Athens: 34.789722222222224 -86.96944444444445 != 34.7897 -86.9694
Demopolis: 32.50944444444445 -87.83722222222222 != 32.5094 -87.8372
Chelsea Alabama: 33.329166666666666 -86.65083333333334 != 33.3292 -86.6508
Pell City Alabama: 33.57083333333333 -86.27388888888889 != 33.5708 -86.2739
City of Northport: 33.25388888888889 -87.59222222222222 != 33.2539 -87.5922
Sand Point: 55.336666666666666 -160.49333333333334 != 55.3367 -160.493
Unalaska Alaska: 53.888888888888886 -166.52722222222224 != 53.8889 -166.527
City of Menlo Park: 37.454166666666666 -122.17861111111111 != 37.4542 -122.179
{Negtemiut|Nightmute}: 60.49138888888889 -164.82611111111112 != 60.4914 -164.826
Fairbanks Alaska: 64.84361111111112 -147.72305555555556 != 64.8436 -147.723
Homer: 59.643055555555556 -151.52583333333334 != 59.6431 -151.526
Ketchikan Alaska: 55.35 -131.67333333333335 != 55.35 -131.673
{Nuniaq|Old Harbor}: 57.19722222222222 -153.30777777777777 != 57.1972 -153.308
Rainier Washington: 46.89083333333333 -122.689722